# Boosting
* refers to any ensemble method that can combine several weak learners into a strong learner.
* Train predictors sequentially, each trying to correct its predecessor.
* AdaBoost(Adaptive boosting) and gradient boosting are the most popular.

## AdaBoost
* The algorithm uses a sequence of classifiers that focuses on misclassified predictions of previous classifiers.
* First trains a base classifier such as a decision tree, make predictions on the training set, and then increaes  
the relative weight of misclassified instances.
* The updated weights are then used to train the second classifier and so on.
* This sequential learning technique looks similar to GD, except that instead of adjusting single predictor's  
parameter to minimize a cost function, it adds predictors to the ensemble, gradually making the model better.
* Drawback: the process cannot be parallelized since each predictor can only be trained after the previous predictor  
has been trained and evaluated.
* See pg201 for the mathematical details.

In [12]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()
X = iris.data[:, 2:] # petal length and width
y = iris.target

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5) 
    # SAMME.R: relies on class probabilities rather than predictions. Generally performs better.
ada_clf.fit(X_train, y_train)
# If the ensemble is overfitting, you can try reducing the number of estimators.

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=1,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                          

## Gradient boosting
* Like Ada, works by sequentially adding predictors to an ensemble, each one correcting its predecessor.
* Instead of adjusting the weights at every iteration, it tries to fit the new predictor to the  
'residual errors' made by the previous predictor.


In [9]:
from sklearn.tree import DecisionTreeRegressor
tree_reg1 = DecisionTreeRegressor(max_depth= 2)
tree_reg1.fit(X, y)

y2 = y- tree_reg1.predict(X) # the 1st residual
tree_reg2 = DecisionTreeRegressor(max_depth= 2)
tree_reg2.fit(X, y2)

y3 = y2 - tree_reg2.predict(X) # the 2nd residual
tree_reg3 = DecisionTreeRegressor(max_depth= 2)
tree_reg3.fit(X, y3)

y_pred = sum(tree.predict(X) for tree in (tree_reg1, tree_reg2, tree_reg3))


In [10]:
# A simpler way:
from sklearn.ensemble import GradientBoostingRegressor
gbrt = GradientBoostingRegressor(max_depth= 2, n_estimators= 3, learning_rate=1.0)
gbrt.fit(X, y)
# if you set the learning rate low, you would need more trees to fit the training set,
# but the predictions will usually generalize better.


GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=1.0, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=3,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### Implementing early stopping
* Implement early stopping to find the optimal number of trees.

In [14]:
# Use staged_predict() which returns an iterator over the predictions made by the ensemble at
# each stage of training (with one tree, two trees, etc)
import numpy as np
from sklearn.metrics import mean_squared_error

gbrt = GradientBoostingRegressor(max_depth= 2, n_estimators= 120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred)
          for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors) + 1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=40,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [16]:
# Or actually stop early by using warm_start=True to allow incremental training.
gbrt = GradientBoostingRegressor(max_depth= 2, warm_start=True)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up = error_going_up + 1 # error_going_up += 1
        if error_going_up ==5 :
            break # early stopping
# GradientBoostingRegressor also supports a subsample hyperparameter.
# If subsample = 0.25, then each tree is trained on 25% of the training instances,
# selected randomly. This technique trades a higher bias for a lower variance and
# the technique is called 'stochastic gradient boosting'

In [17]:
# It is worth noting that an optimized implementation of gradient boosting is available.
# XGboost stands for 'extreme gb'. It is extremely fast, scalable, and portable.
import xgboost
xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)
# It can automatically take care of early stopping
xgb_reg.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds= 2)
y_pred = xgb_reg.predict(X_val)

[14:58:37] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:58:37] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.964838
Will train until validation_0-rmse hasn't improved in 2 rounds.
[1]	validation_0-rmse:0.882443
[2]	validation_0-rmse:0.808653
[3]	validation_0-rmse:0.742666
[4]	validation_0-rmse:0.683755
[5]	validation_0-rmse:0.628232
[6]	validation_0-rmse:0.577881
[7]	validation_0-rmse:0.531613
[8]	validation_0-rmse:0.494354
[9]	validation_0-rmse:0.459171
[10]	validation_0-rmse:0.425955
[11]	validation_0-rmse:0.396254
[12]	validation_0-rmse:0.373307
[13]	validation_0-rmse:0.353204
[14]	validation_0-rmse:0.335701
[15]	validation_0-rmse:0.319195
[16]	validation_0-rmse:0.306033
[17]	validation_0-rmse:0.294694
[18]	validation_0-rmse:0.285129
[19]	valid

## Stacking
* Short for 'stacked generalization'
* Instead of using trivial functions such as hard voting to aggregate the predictions of  
all predictors in an ensemble, we can try to train a model to perform this aggregation.
* When initial predictors come up with their predictions, a 'blender' uses those predictions  
as inputs and makes the final prediction.
* To train a blender, use a hold-out set.  
    1) Training set is split into two subsets. The 1st subset is used to train the predictors  
    in the first layer. (Let's assume that there are 3 predictors)  
    2) the 1st layer predictors then make predictions on the second(held-out) set.  
    This ensures that the predictions are 'clean'.
    3) So for each instance in the hold-out set, there are three predicted values.  
     (made by each of three predictors in the 1st layer)  
    4) This can create a new training set using these predicted values as input features, 
    and keeping the target values. The blender then train on this new training set.
* Can apply a similar process for multilayer stacking ensemble. For 3 layer ensemble (2 predictor, 1 blender)  
need to divide the training set into 3 subsets instead of 2.
* sklearn doesn't support stacking directly.